# 1. A first look at image classification with ConvNets

We will take a look at using convolutional layers inside a neural network. In particular, we will work on classifying X-ray images of lungs as normal or having pneumonia.
This part of the notebook is based in parts on a Kaggle tutorial by [Madhav Mathur](https://www.kaggle.com/madz2000/pneumonia-detection-using-cnn-92-6-accuracy).

If you haven't done so, you will need to install opencv with the following line (note that the import uses `cv2` instead of the full name).

In [ ]:
#pip install opencv-python

We have a number of imports to do. You should be familiar with all of those, except `cv`, `Model`, `Conv2D`, `MaxPool2D`, `ImageDataGenerator`, and `ReduceLROnPlateau` which we introduce throughout the notebook.

In [ ]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPool2D, Flatten, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

## 1.1 Loading data

We use the following dataset, also available on Kaggle: [Kermany, Zhang, Goldbaum (2018)"Labeled Optical Coherence Tomography (OCT) and Chest X-Ray Images for Classification"](https://www.kaggle.com/paultimothymooney/chest-xray-pneumonia). Note that we have somewhat shifted around images from the test to the validation set (the original validation set only contains 16 images).

The function below takes any of the subdirectories (`'train'`, `'val'`, `'test'`), it then
1. Reads in the images (interpreted as being in grayscale), using `cv2`
1. Resizes them according tour our pre-defined `img_size`
1. Reshapes the images to be three-dimension (with one channel instead of three, as we have grayscale images, not RGB images)
1. Normalizes the pixel values to the 0-1 interval
1. Returns the image data and the corresponding label as 0 (normal) or 1 (pneumonia)

In [ ]:
labels = ['NORMAL','PNEUMONIA']
img_size = 200
def load_data(data_dir):
    x_data = [] 
    y_data = []
    for label in labels: 
        path = os.path.join(data_dir, label)
        class_num = labels.index(label)
        for img in [f for f in os.listdir(path) if not f.startswith('.')]:
            try:
                img_arr = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
                resized_arr = cv2.resize(img_arr, (img_size, img_size)) # reshape images to preferred size
                x_data.append(resized_arr)
                y_data.append(class_num)
            except Exception as e:
                print(e)
    x = np.stack(x_data,axis=0).reshape(-1,img_size,img_size,1) # reshape images to 4-dim array
    x = x / 255 # normalize data to 0-1
    y = np.array(y_data,dtype='float32')    
    return x, y

We call the previous function in order to load the three data sets:

In [ ]:
x_train, y_train = load_data('chest_xray/train')
x_val, y_val = load_data('chest_xray/val')
x_test, y_test = load_data('chest_xray/test')

In [ ]:
print(x_train.shape)
print(x_val.shape)
print(x_test.shape)

Let's take a look at the labels on the training data. it seems that there are many more images with pneumonia than without, but the order of magnitude is similar, so this should be okay:

In [ ]:
plt.hist((y_train[y_train==0.0],y_train[y_train==1.0]))
plt.show()

We can also take a look at some examples with and without pneumonia:

In [ ]:
n = 5
plt.figure(figsize=(2*n, 4))
for i in range(n):
    # display normal
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(x_train[np.where(y_train==0.0)[0][i]],cmap='gray')
    plt.title("normal")
    #plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display pneumonia
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(x_train[np.where(y_train==1.0)[0][i]],cmap='gray')
    plt.title("pneumonia")
    #plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

## 1.2 Data augmentation

In order to avoid overfitting, aside from regularization, we can artifically extend our dataset. This is commonly done with image data, especially when we have only few datapoints. The idea is to alter the training data with small transformations to produce different images every time our fitting process calls up the next batch. In this example, we will:

1. Randomly rotate by up to 30 degrees
1. Randomly zoom by up to 20% some
1. Randomly shift images horizontally by up to 10% of their width
1. Randomly shift images vertically by up to 10% of their height
1. Randomly flip images horizontally.

Note that we are not creating a fixed number of data points. Instead, we create an `ImageDataGenerator`, which is fit to our original training data, and generates new images by randomly applying the different operations mentioned above. It keeps giving out images as long as we are fitting the model.

In [ ]:
datagen = ImageDataGenerator(
        rotation_range = 30,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.2, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip = True)  # randomly flip images horizontally

datagen.fit(x_train)

## 1.3 A standard feed-forward network

Let's start by creating a standard feed-forward network with 3 hidden layers and 100 neurons each, as well as a sigmoid output layer (why?) We will use the `Adam` optimizer with a pre-specified learning rate.

In [ ]:
model = Sequential([
    Flatten(input_shape=(img_size,img_size,1)),
    
    Dense(100, activation="relu"),
    
    Dense(100, activation="relu"),
    
    Dense(100, activation="relu"),
    
    Dense(1, activation="sigmoid")
])
model.compile(loss="binary_crossentropy",
              optimizer=tf.keras.optimizers.Adam(5e-4),
              metrics=["accuracy"])
model.summary()

We next fit the model. We will use a new callback, the `ReduceLROnPlateau`, which observes a certain metric and if this metric no longer improves (considering a certain patience), reduces the learning rate by a given factor. Very similar to the learning rate schedules we have seen before, just some different mechanics.

In [ ]:
lr_red_callback = ReduceLROnPlateau(monitor='val_accuracy', patience = 2, verbose=1,factor=0.3, min_lr=1e-6)

Otherwise, this is the usual process, just that we are not directly inputting our training data, but instead we are providing data from the `ImagaDataGenerator`. We specify a `batch_size` as for the usual fitting process, but this also tells the generator how many images to return at any time.

In [ ]:
log = model.fit(datagen.flow(x_train,y_train, batch_size = 32),
                epochs=12,
                validation_data=datagen.flow(x_val,y_val),
                callbacks=[lr_red_callback])

Let's see how we are doing in terms of loss and accuracy:

In [ ]:
def create_plot(log):
    plt.plot(log.history['accuracy'],label = "training accuracy",color='green')
    plt.plot(log.history['loss'],label = "training loss",color='darkgreen')
    plt.plot(log.history['val_accuracy'], label = "validation accuracy",color='grey')
    plt.plot(log.history['val_loss'], label = "validation loss",color='darkblue')
    plt.legend()
    plt.show()
create_plot(log)

The validation accuracy is not fantastic. With a bit of fine-tuning (dropout, batch normalization, a better learning rate, some callbacks), you can reach an accuracy of 90% or so. Not fantastic, but a good start.

Once we are fine with our model, we will evaluate it on the test set

In [ ]:
model.evaluate(x_test, y_test)

Let's also take a look at the confusion matrix:

In [ ]:
predictions = model.predict(x_test)
predictions = predictions.reshape(1,-1)[0]
predictions = (predictions > 0.5)
confusion_matrix(y_test,predictions)

## 1.4 A convolutional neural network

We will now use convolution and max-pooling layers. As is common, we start with a few (large) channels, then decrease the width and height of our layers, but make them increasingly deep.

At the end, we add a dense layer, as well as the sigmoid output from before.

In [ ]:
model_conv = Sequential([
    Conv2D(32, kernel_size=(3,3), strides=(2,2), padding='same', activation="relu", input_shape=(img_size,img_size,1)),
    MaxPool2D((2,2), strides=(2,2), padding='same'),
    
    Conv2D(64, kernel_size=(3,3), strides=(2,2), padding='same', activation="relu"),
    MaxPool2D((2,2), strides=(2,2), padding='same'),
    
    Flatten(),
    Dense(100, activation="relu"),
    
    Dense(1, activation="sigmoid")
])
model_conv.compile(loss="binary_crossentropy",
              optimizer=tf.keras.optimizers.Adam(5e-4),
              metrics=["accuracy"])
model_conv.summary()

Training proceeds exactly as before. Note that, even though we have many less parameters, it tends to take much longer to train convlutional neural networks. Be ready to wait a bit (and this is a very simple network, still!)

In [ ]:
lr_red_callback = ReduceLROnPlateau(monitor='val_accuracy', patience = 2, verbose=1,factor=0.3, min_lr=1e-6)

In [ ]:
log_conv = model_conv.fit(datagen.flow(x_train,y_train, batch_size = 32),
                            epochs=12,
                            validation_data=datagen.flow(x_val,y_val),
                            callbacks=[lr_red_callback])

In [ ]:
create_plot(log_conv)

The accuracy on the validation set looks somewhat better (and also less variable). There is still a lot to be done. Again, dropout and batch normalization can help, as well as adding more layers. The network presented in the Kaggle notebook linked above gets to around 93% accuracy. With much bigger networks, 96-99% should be possible - but we will only use such networks pre-trained, or we could be spending quite some time training.

In [ ]:
model_conv.evaluate(x_test, y_test)

In [ ]:
predictions = model_conv.predict(x_test)
predictions = predictions.reshape(1,-1)[0]
predictions = (predictions > 0.5)
confusion_matrix(y_test,predictions)

### DISCUSSION AND TO DOS

Can you do better? Try adding Dropout and Batch-Normalization layers.

Would you say it is easier or harder to fine-tune a CNN or a normal feed-forward network? Why?








# 2. Non-sequential models with the Functional API

We will now take a first peek at running non-sequential models. An example application is object detection (we need to predict classes **and** bounding boxes). But there are many other sceneraios where you want to tweak your model non-sequentially, e.g., to introduce skip connection (see next week's video).

Here, we will train a model that predicts both normal/pneumonia lungs, as well as the average value of the input pixel images (not too hard to predict, but this is really just to show you how we can use the Functional API of TensorFlow).

Let's first create the secondary y's:

In [ ]:
y2_train = np.mean(x_train,axis=(1,2,3))
y2_val = np.mean(x_val,axis=(1,2,3))
y2_test = np.mean(x_test,axis=(1,2,3))

The Functional API works very similarly to the Sequential API. But instead of having a list of layers, we just create layers and connect them arbitrarily. To do so, we just specify the previous layer that is supposed to flow into the current layer:

In [ ]:
model_input = Input(shape=(img_size, img_size, 1)) # We start with an input layer (we could have multiple inputs, too!)

x = Conv2D(32, kernel_size=(3,3), strides=(2,2), padding='same', activation="relu")(model_input) # We then create a Convolutional layer, which takes the input layer as an input
x = MaxPool2D((2,2), strides=(2,2), padding='same')(x) # Next, we create a Pooling layer that takes the convolutional layer as its input
    
x = Conv2D(64, kernel_size=(3,3), strides=(2,2), padding='same', activation="relu")(x) # As before
x = MaxPool2D((2,2), strides=(2,2), padding='same')(x) # As before
     
x = Flatten()(x)  # As before
x = Dense(100,activation="relu")(x)  # As before

model_output_1 = Dense(1, activation="sigmoid", name = 'output_1')(x) # Now we create an output layer that predicts the class (normal / pneumonia). It uses whatever comes out of the network so far
model_output_2 = Dense(1, activation="sigmoid", name = 'output_2')(x) # We create a second output layer. Note that this does not connect to the other output layer, but directly to the last hidden layer

We have all the same layers defined as before, just with a second output layer. Note that we give a specific name to our output-layers, so we can reference them later!

We combine our layers in a model. We just have ot specify what our inputs are and what our outputs are. The remaining layers are added based on the structure above!

In [ ]:
model_func = Model(inputs = [model_input], outputs=[model_output_1, model_output_2])

See for yourself:

In [ ]:
model_func.summary()

Let's now compile our non-sequential model. We need to define our losses and metrics for each of our outputs! This is why we gave the output layers specific names, so we can use this here. The rest is as before:

In [ ]:
model_func.compile(loss={'output_1':'binary_crossentropy',
                         'output_2':'mean_squared_error'},
                   loss_weights = [1,0.01],
                   metrics = {'output_1':'accuracy',
                             'output_2':'mean_squared_error'},
                   optimizer=tf.keras.optimizers.Adam(5e-4))

Similar for fitting the model: We have to make clear that there are two different y values that need to be predicted. We will run the model only for a few epochs, to see how it works.

In [ ]:
log_func = model_func.fit(x=x_train,y=[y_train,y2_train],
                            epochs=3,
                            validation_data=(x_val,[y_val,y2_val]))

As always, we can evaluate the model:

In [ ]:
model_func.evaluate(x_test, [y_test,y2_test])

And we can make predictions. We just have to note that two outputs are being predicted, the labels and the average values. But we can simply use list-indices to get the right ones.

In [ ]:
predictions = model_func.predict(x_test)
predictions_label = predictions[0].reshape(1,-1)[0]
predictions_label = (predictions_label > 0.5)
confusion_matrix(y_test,predictions_label)

In [ ]:
predictions_avg = predictions[1]
predictions_avg